In [1]:
import re
import nltk
import json
import pymorphy2
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smoot\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smoot\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
with open('./setting.json', 'r', encoding='utf-8') as f_setting:
    setting = json.load(f_setting)

In [4]:
def text_tokenizer(text: str):
    morph = pymorphy2.MorphAnalyzer(lang='ru')
    
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    
    tokenize = [token for token in nltk.word_tokenize(text)]
    remove_stopW = [clear_txt for clear_txt in tokenize if clear_txt not in setting['stopwords']]
    morph_text = [morph.parse(m_text)[0].normal_form for m_text in remove_stopW]
    
    return morph_text

In [10]:
text = ['Машинист, Иванов Иван Иванович, электросостава №, следую по маршруту А-Б', 'Машинист,Иванов Иван Иванович продолжайте следование по маршруту А-Б']
text_corpus = [text_tokenizer(i) for i in text]

In [11]:
ohe_model = CountVectorizer(tokenizer=text_tokenizer).fit_transform(text)
ohe = ohe_model.toarray()
ohe

array([[1, 1, 1, 1, 1, 1, 0, 0, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]], dtype=int64)

In [12]:
cosine_similarity(ohe_model)

array([[1.  , 0.75],
       [0.75, 1.  ]])

In [13]:
def cosine_sim_vector(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    
    return cosine_similarity(vec1, vec2)[0][0]

In [14]:
print(f"Текста схожи на: {round(cosine_sim_vector(ohe[0], ohe[1]) * 100, 2)}%")

Текста схожи на: 75.0%


## Парсинг текста

In [20]:
text = input("Введите текст, согласно шаблону: <t> сообщение\n ")

Введите текст, согласно шаблону: <t> сообщение
  <s>Машинист поезда №125 следует по маршруту №39 <s> Машинист поезда №125 следование по маршруту №39 подтверждаю


In [24]:
textArr = text.split('<s>')
textArr

['',
 'Машинист поезда №125 следует по маршруту №39 ',
 ' Машинист поезда №125 следование по маршруту №39 подтверждаю']

In [26]:
ohe_model = CountVectorizer(tokenizer=text_tokenizer).fit_transform(textArr)
ohe = ohe_model.toarray()
ohe

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 1, 1, 1, 1, 0]], dtype=int64)

In [27]:
print(f"Текста схожи на: {round(cosine_sim_vector(ohe[1], ohe[2]) * 100, 2)}%")

Текста схожи на: 77.15%
